In [30]:
# Librerías principales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.signal as signal
import pywt  # wavelet
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from pathlib import Path


In [36]:
# --- Cargar datos principales ---
# Establecer rutas relativas
DATA_DIR = Path('../data/raw')

# Cargar TSM y Chla
tsm = pd.read_excel(DATA_DIR / 'Base de datos TSM consultoria estadística.xlsx')
chla = pd.read_excel(DATA_DIR / 'Base de datos Chla consultoria estadística.xlsx')
coordenadas = pd.read_csv(DATA_DIR / 'Coordenadas zona costera occidental GC.csv')

# Cargar ONI (estaciones móviles)
oni = pd.read_csv(DATA_DIR / 'oni-Cold & Warm Episodes by Season.csv - Sheet1.csv')

# --- Limpiar nombres de columnas ---
# Eliminar espacios y símbolos de grados °
tsm.columns = tsm.columns.str.strip().str.replace('°', '', regex=False)
chla.columns = chla.columns.str.strip().str.replace('°', '', regex=False)

# Verificar nombres de columnas
print("Columnas TSM:", tsm.columns.tolist())
print("Columnas Chla:", chla.columns.tolist())

# --- Limpiar archivo ONI ---
# Eliminar filas basura (donde Year no sea número)
oni = oni[oni['Year'].apply(lambda x: str(x).isdigit())].copy()
oni['Year'] = oni['Year'].astype(int)

# Formato largo
oni_long = oni.melt(id_vars='Year', var_name='Season', value_name='ONI')

# Mapeo estaciones móviles a meses
season_to_month = {
    'DJF': 1, 'JFM': 2, 'FMA': 3, 'MAM': 4,
    'AMJ': 5, 'MJJ': 6, 'JJA': 7, 'JAS': 8,
    'ASO': 9, 'SON': 10, 'OND': 11, 'NDJ': 12
}
oni_long['Month'] = oni_long['Season'].map(season_to_month)

# Crear columna de fecha
oni_long['Fecha'] = pd.to_datetime(dict(year=oni_long['Year'],
                                        month=oni_long['Month'],
                                        day=15))

# Limpiar y ordenar
oni_long = oni_long[['Fecha', 'ONI']].dropna().sort_values('Fecha').reset_index(drop=True)

# --- Alinear FECHAS para hacer merge ---
# Convertir FECHA a periodo mensual (año-mes) para tsm y chla
tsm['FECHA'] = pd.to_datetime(tsm['FECHA']).dt.to_period('M').dt.to_timestamp()
chla['FECHA'] = pd.to_datetime(chla['FECHA']).dt.to_period('M').dt.to_timestamp()
oni_long['Fecha'] = oni_long['Fecha'].dt.to_period('M').dt.to_timestamp()

# --- Merge de datos ---
merged_tsm = pd.merge(tsm, oni_long, left_on='FECHA', right_on='Fecha', how='inner')
merged_chla = pd.merge(chla, oni_long, left_on='FECHA', right_on='Fecha', how='inner')

# Verificar
print("TSM merged:", merged_tsm.shape)
print("Chla merged:", merged_chla.shape)

# --- Ejemplo de correlación en una estación (ya normalizada el nombre) ---
print("Correlación TSM (Est 1) - ONI:", merged_tsm['Est 1'].corr(merged_tsm['ONI']))
print("Correlación Chl a (Est 1) - ONI:", merged_chla['Est 1'].corr(merged_chla['ONI']))

Columnas TSM: ['ORDEN', 'FECHA', 'AÑO', 'MES', 'ESTACIÓN DEL AÑO', 'EVENTO INTERANUAL', 'Est 1', 'Est 2', 'Est 3', 'Est 4', 'Est 5', 'Est 6', 'Est 7', 'Est 8', 'Est 9', 'Est 10', 'Est 11', 'Est 12', 'Est 13', 'Est 14', 'Est 15', 'Est 16', 'Est 17']
Columnas Chla: ['ORDEN', 'FECHA', 'AÑO', 'MES', 'ESTACIÓN DEL AÑO', 'EVENTO INTERANUAL', 'Est 1', 'Est 2', 'Est 3', 'Est 4', 'Est 5', 'Est 6', 'Est 7', 'Est 8', 'Est 9', 'Est 10', 'Est 11', 'Est 12', 'Est 13', 'Est 14', 'Est 15', 'Est 16', 'Est 17']
TSM merged: (446, 25)
Chla merged: (254, 25)
Correlación TSM (Est 1) - ONI: 0.1229546176418447
Correlación Chl a (Est 1) - ONI: -0.2785677288568203


In [32]:
# Lista de estaciones
estaciones = [col for col in tsm.columns if col.startswith("Est ")]

# Crear DataFrame vacío
correlaciones = pd.DataFrame(columns=["Estación", "Corr_TSM_ONI", "Corr_Chla_ONI"])

# Calcular correlación para cada estación
for est in estaciones:
    corr_tsm = merged_tsm[est].corr(merged_tsm["ONI"])
    corr_chla = merged_chla[est].corr(merged_chla["ONI"])
    correlaciones = pd.concat([
        correlaciones,
        pd.DataFrame([{
            "Estación": est,
            "Corr_TSM_ONI": round(corr_tsm, 3),
            "Corr_Chla_ONI": round(corr_chla, 3)
        }])
    ], ignore_index=True)

# Mostrar tabla
print(correlaciones)


   Estación  Corr_TSM_ONI  Corr_Chla_ONI
0     Est 1         0.123         -0.279
1     Est 2         0.116         -0.207
2     Est 3         0.107         -0.170
3     Est 4         0.106         -0.191
4     Est 5         0.111         -0.085
5     Est 6         0.106         -0.113
6     Est 7         0.104         -0.078
7     Est 8         0.097         -0.004
8     Est 9         0.104         -0.054
9    Est 10         0.115          0.036
10   Est 11         0.108          0.097
11   Est 12         0.088          0.038
12   Est 13         0.069         -0.022
13   Est 14         0.053          0.024
14   Est 15         0.057         -0.065
15   Est 16         0.046         -0.078
16   Est 17         0.036         -0.026


C:\Users\angel\AppData\Local\Temp\ipykernel_972\3672781692.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  correlaciones = pd.concat([


In [35]:
# Guardar como CSV
correlaciones.to_csv("../data/interim/AABR_Notebooks/correlaciones_estaciones_ONI.csv", index=False)